# [Bonus Tasks](svm_segm.ipynb)

In [96]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [97]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [98]:
patch_sizes = (32, 32)

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [99]:
def create_data_matrix(img) :
    patches = skimage.util.view_as_blocks(img, block_shape = patch_sizes)
    num_patches = patches.shape[0] * patches.shape[1]
    X = patches.reshape(num_patches, -1)
    
    return np.array(X, dtype=np.int32)

In [100]:
skimage.util.view_as_blocks(img, block_shape = patch_sizes).shape


(32, 42, 32, 32)

**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [101]:
def create_gt_labels_vector(img) :
    patches = skimage.util.view_as_blocks(img, block_shape = patch_sizes)
    num_patches = patches.shape[0] * patches.shape[1]
    
    #y = np.zeros(num_patches, dtype = np.int32)
    y = []

    for i in range(patches.shape[0]) :
        for j in range(patches.shape[1]) :
            patch = patches[i, j]

            if np.mean(patch) > 0.5 :
                y.append(1)
            
            elif np.mean(patch) == 0 :
                y.append(-1)
            
            else :
                y.append(0)

    return np.array(y, dtype=np.int32)

**[Task 1.3.]()** Create the SVM classifier:

In [102]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [103]:
dna_33 = plt.imread('data/NIH3T3/im/dna-33.png')
dna_44 = plt.imread('data/NIH3T3/im/dna-44.png')

X_33 = create_data_matrix(dna_33)
X_44 = create_data_matrix(dna_44)

#print(X_33)

**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [104]:
ground_33 = plt.imread('data/NIH3T3/gt/33.png')
ground_44 = plt.imread('data/NIH3T3/gt/44.png')

Y_33 = create_gt_labels_vector(ground_33)
Y_44 = create_gt_labels_vector(ground_44)

**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

In [105]:
X_valid = np.concatenate([X_33[Y_33 != 0], X_44[Y_44 != 0]])
Y_valid = np.concatenate([Y_33[Y_33 != 0], Y_44[Y_44 != 0]])

**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

In [106]:
clf.fit(X_valid, Y_valid)    

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced', gamma=0.1))])

**[Task 1.4.]()** Implement the function `predict_image`:

In [127]:
def predict_image(img) :
    X = create_data_matrix(img)
    prediction = clf.predict(X)

    result = np.zeros(img.shape, dtype=bool)
    
    patches = skimage.util.view_as_blocks(result, block_shape = patch_sizes)
    '''num_patches = patches.shape[0] * patches.shape[1]
    X = patches.reshape(num_patches, -1)'''
    
    t = np.full(patch_sizes, True, dtype=bool)

    for i in range(prediction.shape[0]) :
        patches[i] = t
    
    result = np.reshape(patches, img.shape)

    return result


In [108]:
X = create_data_matrix(img)
perdiction = clf.predict(X)

Test your implementation:

In [128]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

plt.imshow(seg, "gray")

'''
plt.figure()
plt.imshow(blend_result(img, seg))
'''


IndexError: index 32 is out of bounds for axis 0 with size 32

**[Task 1.5.]()** Perform batch processing:

In [110]:
seq = [28, 29, 33, 44, 46, 49]

all = []

for i in seq :
    img = plt.imread(f'data/NIH3T3/im/dna-{i}.png')
    ground = plt.imread(f'data/NIH3T3/gt/{i}.png')

    binground = ground > 0.5
    binimg = predict_image(img)
    
    dice = 2 * (binimg * binground).sum() / (binimg.sum() + binground.sum())

    all.append(dice)

    print('image: ' + str(i) + ', Dice: ' + str(round(dice, 2)))

print()
print('Average Dice: ' + str(round(sum(all)/len(all), 2)))



image: 28, Dice: 0.0
image: 29, Dice: 0.0
image: 33, Dice: 0.0
image: 44, Dice: 0.0
image: 46, Dice: 0.0
image: 49, Dice: 0.0

Average Dice: 0.0
